#### Precipitation and Air Temperature data were downloaded from
- NASA's [Daymet](https://daymet.ornl.gov/) using coordinates from the Clemson University Pee Dee Research Center
- latitude: 34.289	
- longitude: -79.737

#### Wind and humidity data were downloaded from
- Climate [Engine](https://clim-engine.appspot.com/climateEngine) using coordinates from the Clemson Pee Dee Research and Education Center
- latitude: 34.289	
- longitude: -79.737

#### These are more precise environmental data which will be used in combination with the trait [data](https://github.com/MagicMilly/for-data-publication/tree/main/data/processed) from Brenton et al. 2016 at the Clemson University Pee Dee Research Center

In [ ]:
import datetime
import numpy as np
import pandas as pd

In [ ]:
# returns vpd in KPa

def calculate_vpd_kpa(temp_avg_c, rh_avg):
    es = (6.11 * np.exp((2500000/461) * (1/273 - 1/(273 + temp_avg_c))))
    vpd = (((100 - rh_avg)/1000) * es)
    return vpd

### A. Daymet Download

In [ ]:
daymet_0 = pd.read_csv('data/raw/clemson/clemson_raw_daily_weather.csv')
# print(daymet_0.shape)
# daymet_0.head()

### Columns from Daymet Download
- year
- day of year
- dayl - daylength, seconds per day
- precipitation in mm
- srad - shortwave radiation (can be used with dayl to calculate total daily radiation)
- swe - snow water equivalent
- air temperature max in C
- air temperature min in C
- vp - daily average partial pressure of water vapor, in Pascals

#### Add Dates

In [ ]:
date_range_2014 = pd.date_range(start='2014-01-01', end='2014-12-31')

In [ ]:
daymet_1 = daymet_0.copy()
daymet_1['date'] = date_range_2014
# print(daymet_1.shape)
# daymet_1.tail()

#### Convert vapor pressure to Kilopascals
will be calculating vapor pressure deficit in KPa

In [ ]:
daymet_2 = daymet_1.copy()
daymet_2['vapor_pressure_kpa'] = daymet_2['vp (Pa)'] / 1000
# daymet_2.sample(n=5)

In [ ]:
# drop original vapor pressure column

daymet_3 = daymet_2.drop(labels='vp (Pa)', axis=1)
# daymet_3.tail(3)

#### Rename Columns

In [ ]:
daymet_4 = daymet_3.rename({'yday': 'day_of_year', 'dayl (s)': 'day_length_seconds', 'prcp (mm/day)': 'precip_mm',
                            'srad (W/m^2)': 'shortwave_rad', 'swe (kg/m^2)': 'snow_water_equivalent', 
                            'tmax (deg c)': 'temp_max_c', 'tmin (deg c)': 'temp_min_c', 
                            'vp (Pa)': 'vapor_pressure_kpa'}, axis=1)

# print(daymet_4.shape)
# daymet_4.tail()

### B. Climate Engine Download
- rh_n - dataset with relative humidity values
- wind_n - dataset with wind values
    - Assumption that these are average wind speed values per day

In [ ]:
rh_0 = pd.read_csv('data/raw/clemson/clemson_rh_2014.csv')
# print(rh_0.shape)
# rh_0.head()

#### Rename Columns

In [ ]:
rh_1 = rh_0.rename({'(%) Min Rel. Humidity (gridMET), -79.7370E,34.2890N ,2014-01-01 to 2014-12-31': 'min_rh',
                   '(%) Max Rel. Humidity (gridMET), -79.7370E,34.2890N ,2014-01-01 to 2014-12-31': 'max_rh'}, axis=1)

# print(rh_1.shape)
# rh_1.head()

#### Change `DateTime` values to datetime objects

In [ ]:
date_range = pd.to_datetime(rh_1.DateTime)
# date_range

In [ ]:
rh_2 = rh_1.copy()
rh_2['date'] = date_range
# print(rh_2.dtypes)
# rh_2.head()

#### Drop DateTime Column

In [ ]:
rh_3 = rh_2.drop('DateTime', axis=1)
# print(rh_3.shape)
# rh_3.tail()

In [ ]:
wind_0 = pd.read_csv('data/raw/clemson/climate_engine_wind_speed_2014.csv')
# print(wind_0.shape)
# wind_0.head()

#### Rename Column

In [ ]:
wind_1 = wind_0.rename({'(m/s) Wind Speed (gridMET) at -79.7370E,34.2890N, 2014-01-01 to 2014-12-31': 'wind_speed_ms'},
                      axis=1)

# print(wind_1.shape)
# wind_1.head()

#### New Column with datetime object

In [ ]:
date_range = pd.to_datetime(wind_1.DateTime)

wind_2 = wind_1.copy()
wind_2['date'] = date_range

# print(wind_2.shape)
# wind_2.tail()

#### Drop DateTime Column

In [ ]:
wind_3 = wind_2.drop('DateTime', axis=1)
# wind_3.head()

#### Merge DataFrames

In [ ]:
temp_rh_wind_0 = daymet_5.merge(rh_3, how='outer', right_on='date', left_on='date').merge(wind_3, how='outer',
                                                                                    left_on='date', right_on='date')

# print(temp_rh_wind_0.shape)
# temp_rh_wind_0.head()

#### Slice for season dates
- Planting: 2014-05-06, Day of Year 126
- Latest date in Clemson trait data: 2014-10-15, Day of Year 288

In [ ]:
temp_rh_wind_1 = temp_rh_wind_0.loc[(temp_rh_wind_0['day_of_year'] >= 126) & (temp_rh_wind_0['day_of_year'] <= 288)]
# print(temp_rh_wind_1.shape)
# temp_rh_wind_1.head(3)

#### Add cumulative precipitation column

In [ ]:
temp_rh_wind_2 = temp_rh_wind_1.copy()

temp_rh_wind_2['cum_precip_mm'] = np.cumsum(temp_rh_wind_2.precip_mm)
# print(temp_rh_wind_2.shape)
# temp_rh_wind_2.tail(10)

#### Add growing degree days

In [ ]:
temp_rh_wind_3 = temp_rh_wind_2.copy()
temp_rh_wind_3['daily_gdd'] = (((temp_rh_wind_3['temp_max_c'] + temp_rh_wind_3['temp_min_c'])) / 2) - 10
# print(temp_rh_wind_3.shape)
# temp_rh_wind_3.head()

In [ ]:
temp_rh_wind_4 = temp_rh_wind_3.copy()
temp_rh_wind_4['gdd'] = np.rint(np.cumsum(temp_rh_wind_4['daily_gdd']))
# temp_rh_wind_4.tail()

In [ ]:
# check that no daily gdd values are negative

# temp_rh_wind_4.loc[temp_rh_wind_4['daily_gdd'] < 0]

In [ ]:
# drop daily_gdd column

temp_rh_wind_5 = temp_rh_wind_4.drop('daily_gdd', axis=1)

#### Add average temperature column

In [ ]:
temp_rh_wind_6 = temp_rh_wind_5.copy()
temp_rh_wind_6['temp_avg_c'] = temp_rh_wind_6[['temp_max_c', 'temp_min_c']].mean(axis=1)
# print(temp_rh_wind_6.shape)
# temp_rh_wind_6.head(3)

#### Add average relative humidity

In [ ]:
temp_rh_wind_7 = temp_rh_wind_6.copy()
temp_rh_wind_7['avg_rh'] = temp_rh_wind_7[['min_rh', 'max_rh']].mean(axis=1)
# print(temp_rh_wind_7.shape)
# temp_rh_wind_7.sample(n=3)

#### Add vapor pressure deficit in Kilopascals

In [ ]:
temp_rh_wind_8 = temp_rh_wind_7.copy()
temp_rh_wind_8['vpd_kpa'] = calculate_vpd_kpa(temp_rh_wind_8['temp_avg_c'], temp_rh_wind_8['avg_rh'])
# print(temp_rh_wind_8.shape)
# temp_rh_wind_8.sample(n=5)

#### Round all floats to two decimal points

In [ ]:
cols_to_round = ['day_length_seconds', 'shortwave_rad', 'temp_max_c', 'temp_min_c', 'vapor_pressure_kpa', 'min_rh', 'max_rh',
                'wind_speed_ms', 'temp_avg_c', 'avg_rh', 'vpd_kpa']

temp_rh_wind_9 = temp_rh_wind_8.copy()
temp_rh_wind_9[cols_to_round] = temp_rh_wind_9[cols_to_round].round(2)
print(temp_rh_wind_9.shape)
temp_rh_wind_9.tail(3)

#### Write to CSV

In [ ]:
temp_rh_wind_9.to_csv('data/weather/clemson_weather.csv', index=False)